In [2]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

Defaulting to user installation because normal site-packages is not writeable
  Using cached boto3-1.28.59-py3-none-any.whl (135 kB)
  Using cached awscli-1.29.59-py3-none-any.whl (4.3 MB)
  Using cached botocore-1.31.59-py3-none-any.whl (11.2 MB)
  Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
     |████████████████████████████████| 670 kB 9.8 MB/s eta 0:00:01
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-1.26.17-py2.py3-none-any.whl (143 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.0-py2.py3-none-any.whl (83 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not curren

In [3]:
%pip install --quiet "faiss-cpu>=1.7,<2" "pypdf>=3.8,<4"
%pip install sqlalchemy
%pip install langchain

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 124 kB 5.5 MB/s eta 0:00:01
  Using cached pydantic-2.4.2-py3-none-any.whl (395 kB)
     |████████████████████████████████| 15.7 MB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 45.9 MB/s eta 0:00:01
  Using cached annotated_types-0.5.0-py3-none-any.whl (11 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms.bedrock import Bedrock
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

PydanticUserError: If you use `@root_validator` with pre=False (the default) you MUST specify `skip_on_failure=True`. Note that `@root_validator` is deprecated and should be replaced with `@model_validator`.

For further information visit https://errors.pydantic.dev/2.4/u/root-validator-pre-skip

In [ ]:
import json
import os
import sys
import boto3
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww
# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
os.environ["AWS_DEFAULT_REGION"] = "us-west-2" # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::195364414018:role/Crossaccountbedrock"
bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

In [ ]:
def OLD_get_llm():
    model_kwargs = { #AI21
    "maxTokens": 1024,
    "temperature": 0,
    "topP": 0.5,
    "stopSequences": ["Human:"],
    "countPenalty": {"scale": 0 },
    "presencePenalty": {"scale": 0 },
    "frequencyPenalty": {"scale": 0 }
    }
    llm = Bedrock(
    credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to 
    region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the defaul
    endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
    model_id="ai21.j2-ultra-v1", #set the foundation model
    model_kwargs=model_kwargs) #configure the properties for Claude
    return llm

In [ ]:
def get_llm():
    llm = Bedrock(
        model_id="anthropic.claude-v2",
        client=bedrock_runtime,
        model_kwargs={"max_tokens_to_sample": 1000},
    )
    return llm

In [ ]:
def OLD_get_index(): 
    #creates and returns an in-memory vector store to be used in the 
    embeddings = BedrockEmbeddings(
    credentials_profile_name=os.environ.get("BWB_PROFILE_NAME"), #sets the profile name to 
    region_name=os.environ.get("BWB_REGION_NAME"), #sets the region name (if not the defaul
    endpoint_url=os.environ.get("BWB_ENDPOINT_URL"), #sets the endpoint URL (if necessary)
    ) #create a Titan Embeddings client
    pdf_path = "2022-Shareholder-Letter.pdf" #assumes local PDF file with this name
    loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
    separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sent
    chunk_size=1000, #divide into 1000-character chunks using the separators above
    chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
    index_creator = VectorstoreIndexCreator( #create a vector store factory
    vectorstore_cls=FAISS, #use an in-memory vector store for demo purposes
    embedding=embeddings, #use Titan embeddings
    text_splitter=text_splitter, #use the recursive text splitter
    )
    index_from_loader = index_creator.from_loaders([loader]) #create an vector store index
    return index_from_loader #return the index to be cached by the client app


In [ ]:
def get_index(): #creates and returns an in-memory vector store to be used in the appl
    embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_runtime)
    pdf_path = "polic_docs/Policy_Certificate_Multiple_vehicle.pdf" #assumes local PDF file with this name
    loader = PyPDFLoader(file_path=pdf_path) #load the pdf file
    text_splitter = RecursiveCharacterTextSplitter( #create a text splitter
    separators=["\n\n", "\n", ".", " "], #split chunks at (1) paragraph, (2) line, (3) sent
    chunk_size=1000, #divide into 1000-character chunks using the separators above
    chunk_overlap=100 #number of characters that can overlap with previous chunk
    )
    index_creator = VectorstoreIndexCreator( #create a vector store factory
    vectorstore_cls=FAISS, #use an in-memory vector store for demo purposes
    embedding=embeddings, #use Titan embeddings
    text_splitter=text_splitter, #use the recursive text splitter
    )
    index_from_loader = index_creator.from_loaders([loader]) #create an vector store index
    return index_from_loader #return the index to be cached by the client app


In [ ]:
def get_memory(): #create memory for this chat session
    memory = ConversationBufferWindowMemory(memory_key="chat_history", return_messages=True)
    return memory
                                            
def get_rag_chat_response(input_text, memory, index): #chat client function
    llm = get_llm()
    conversation_with_retrieval = ConversationalRetrievalChain.from_llm(llm, retriever=index.vectorstore.as_retriever(), memory=memory)
    chat_response = conversation_with_retrieval({"question": input_text}) #pass the user me
    return chat_response['answer']

In [ ]:
## TESTING

In [ ]:
gpt_index = get_index()
gpt_memory = get_memory()

In [ ]:
ques =" Can i add my spouce as a new driver to my policy ?"
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)

In [ ]:
ques =" How about my aunt ?"
get_rag_chat_response(input_text=ques,index=gpt_index, memory=gpt_memory)